In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.utils.data as data
import torch.optim as optim
import numpy as np

from datasets import load_dataset
from src.landscape import Landscape
from src.trainer import Trainer
from src.plot import Plot

In [2]:
def pre_process(example):
    arr = np.reshape(example["input"], -1)
    example["input"] = arr
    return example

In [3]:
mnist = load_dataset("mnist", trust_remote_code=True)
train, test = mnist.get("train"), mnist.get("test")

In [4]:
train.set_format(type="numpy", columns=["image", "label"])
test.set_format(type="numpy", columns=["image", "label"])
train = train.rename_column("image", "input")
test = test.rename_column("image", "input")
train = train.map(pre_process, num_proc=4)
test = test.map(pre_process, num_proc=4)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
train_inputs = torch.from_numpy(train["input"]).float().squeeze() / 255.0
test_inputs = torch.from_numpy(test["input"]).float().squeeze() / 255.0
train_labels = torch.from_numpy(train["label"]).long()
test_labels = torch.from_numpy(test["label"]).long()

In [6]:
train_dataset = data.TensorDataset(train_inputs, train_labels)
test_dataset = data.TensorDataset(test_inputs, test_labels)

In [7]:
class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(28 * 28, 512)
        self.drop_1 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(512, 512)
        self.drop_2 = nn.Dropout(0.25)
        self.linear_3 = nn.Linear(512, 10)

    def forward(self, x):
        x = f.relu(self.drop_1(self.linear_1(x)))
        x = f.relu(self.drop_2(self.linear_2(x)))
        return f.relu(self.linear_3(x))


In [8]:
model = Model()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()
batch_size = 256

In [9]:
train_loader = data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
    pin_memory=True,
)

test_loader = data.DataLoader(
    test_dataset,
    batch_size=len(test_dataset),
    shuffle=True,
    drop_last=False,
    pin_memory=True,
)

In [10]:
trainer = Trainer(model, optimizer, loss_fn, write=True, metric_path="./metrics.h5", param_path="./params.pt", traj_path="./traj.pt")

In [11]:
metrics, trajectory = trainer.train(train_loader, test_loader, epochs=50, device=device, print_every=10)

Training started...
Epoch 10 complete, train loss: 0.7366, train acc: 0.76, test loss: 0.6414, test acc: 0.79
Epoch 20 complete, train loss: 0.3491, train acc: 0.90, test loss: 0.2787, test acc: 0.92
Epoch 30 complete, train loss: 0.2775, train acc: 0.92, test loss: 0.2217, test acc: 0.94
Epoch 40 complete, train loss: 0.2329, train acc: 0.93, test loss: 0.1842, test acc: 0.95
Epoch 50 complete, train loss: 0.2014, train acc: 0.94, test loss: 0.1588, test acc: 0.95
Training complete
Writing metrics to: ./metrics.h5
Writing parameters to: ./params.pt
Writing trajectory to: ./traj.pt
Metrics and weights written


In [12]:
model = Model()
landscape = Landscape.from_files(model, loss_fn, param_path="./params.pt", traj_path="./traj.pt")

In [13]:
(X, Y, Z), trajectory = landscape.create_landscape(test_loader, mode="pca", print_every=25)

Iteration: 25, loss: 2.3441
Iteration: 50, loss: 2.3311
Iteration: 75, loss: 2.3220
Iteration: 100, loss: 2.3171
Iteration: 125, loss: 2.3148
Iteration: 150, loss: 2.3131
Iteration: 175, loss: 2.3101
Iteration: 200, loss: 2.3040
Iteration: 225, loss: 2.2936
Iteration: 250, loss: 2.2788
Iteration: 275, loss: 2.2606
Iteration: 300, loss: 2.2412
Iteration: 325, loss: 2.2227
Iteration: 350, loss: 2.2065
Iteration: 375, loss: 2.1935
Iteration: 400, loss: 2.1841
Iteration: 425, loss: 2.1799
Iteration: 450, loss: 2.1838
Iteration: 475, loss: 2.2004
Iteration: 500, loss: 2.2328
Iteration: 525, loss: 2.2835
Iteration: 550, loss: 2.3521
Iteration: 575, loss: 2.4378
Iteration: 600, loss: 2.5388
Iteration: 625, loss: 2.6536
Writing to f./landscape.h5
./landscape.h5 written


In [3]:
plot = Plot.from_files(mesh_path="./landscape.h5")

In [14]:
plot.plot_surface_3D(cmap="Oranges", file_path="./loss-landscape.png")

In [11]:
plot.plot_contour(cmap="YlGnBu", levels=35, plot_trajectory=True, file_path="./loss-contour.png")

In [12]:
plot.animate_contour(cmap="YlGnBu", levels=35,  file_path="./animated-contour.gif")